# Data Cleaning for Analysis

##Overview
This Jupyter notebook outlines the process of cleaning the dataset in preparation for analysis. Through a series of steps and functions, the notebook addresses various data quality issues such as missing values, outliers, incorrect data types, and duplicates. By ensuring that the dataset is clean and well-structured, it sets the stage for accurate and reliable data analysis.

##Description
The cleaning process involves several key steps, including handling rows with numeric values in name fields, creating a full name column, reordering columns to a desired order, flagging outliers, replacing negative values, converting data types, removing duplicates, and handling empty fields. Each step is implemented as a separate function, allowing for modular and efficient data cleaning.

##Purpose
The purpose of this notebook is to transform raw, messy data into a clean and structured format suitable for analysis. By addressing common data quality issues, such as outliers and missing values, the notebook ensures that subsequent analyses are based on reliable and accurate data. This enhances the overall quality and integrity of the analysis results.

### Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.mode.copy_on_write = True

### Functions

In [3]:
def handle_empty_fields(df, value=np.nan):
    """
    Handle empty fields in a DataFrame by filling them with a specified value.

    Parameters:
        df (pandas.DataFrame): The DataFrame to be cleaned.
        value: The value to fill empty fields with (default is NaN).

    Returns:
        pandas.DataFrame: The DataFrame with empty fields filled with the specified value.

    """
    # Find columns with empty fields
    empty_columns = df.columns[df.isnull().any()]

    if empty_columns.any():
        print("Empty fields found in the following columns:")
        for col in empty_columns:
            print(col)

    # Fill empty fields with specified value
    return df.fillna(value)

In [4]:
def remove_duplicates(df):
    """
    Remove duplicate rows from a DataFrame.

    Parameters:
        df (pandas.DataFrame): The DataFrame to be cleaned.

    Returns:
        pandas.DataFrame: The cleaned DataFrame without duplicate rows.
        int: The number of duplicate rows removed.

    """
    duplicate_rows = df.duplicated().sum()
    print(f"Duplicate rows: {duplicate_rows}")

    if duplicate_rows > 0:
        df = df.drop_duplicates()
        print("Duplicate rows removed.")
    else:
        print("No duplicate rows found.")

    return df

In [5]:
def convert_data_types(df, data_types):
    """
    Convert DataFrame columns to the correct data types.

    Parameters:
        df (pandas.DataFrame): The DataFrame to be converted.

    Returns:
        pandas.DataFrame: The DataFrame with correct data types.

    """

    # Convert columns to correct data types
    for col, dtype in data_types.items():
        if col in df.columns:
            df[col] = df[col].astype(dtype)

    return df

In [6]:
def replace_negative_values(df, columns):
    """
    Replace negative values in specific columns with linear interpolation.

    Parameters:
        df (pandas.DataFrame): The DataFrame to be cleaned.

    Returns:
        pandas.DataFrame: The DataFrame with negative values replaced by linear interpolation.

    """

    for column in columns:
        if (df[column] < 0).any():
            print(f"Negative values found in {column}")

            # Identify rows with negative values
            negative_rows = df[df[column] < 0]
            if not negative_rows.empty:
                print(f"Negative values found in {column}:")
                print(negative_rows[[column]].head())

            # Replace negative values with linear interpolation
            negative_indices = df.index[df[column] < 0]
            df.loc[negative_indices, column] = df.loc[negative_indices, column].interpolate(method='linear')

    return df

In [7]:
def flag_outliers(df, include_columns, threshold=0.99, outlier_flag_column='is_outlier'):
    """
    Flag outliers in specified numeric columns of a DataFrame by assigning a separate category or label.

    Parameters:
        df (pandas.DataFrame): The DataFrame to be cleaned.
        include_columns (list): List of column names to include in outlier detection.
        threshold (float): Threshold for outliers (default is 0.99, i.e., 99th percentile).
        outlier_flag_column (str): Name of the column to indicate whether each row is an outlier (default is 'is_outlier').

    Returns:
        pandas.DataFrame: DataFrame with outliers flagged.

    """
    # Initialize the outlier flag column with False
    df[outlier_flag_column] = False

    # Identify outliers for each specified column
    for column in include_columns:
        percentile_value = df[column].quantile(threshold)
        df.loc[df[column] > percentile_value, outlier_flag_column] = True

    return df

In [8]:
def handle_rows_with_numbers(df, action='remove_numbers'):
    """
    Handle rows where 'first_name' or 'last_name' columns contain numbers.

    Parameters:
        df (pandas.DataFrame): The DataFrame to be cleaned.
        action (str): The action to take for resolving the issue (default is 'remove_numbers').
                      Valid options are 'remove_numbers' to remove numbers from names,
                      'remove' to remove rows with numbers, or 'replace' to replace numeric values.

    Returns:
        pandas.DataFrame: DataFrame with the specified action taken to resolve the issue.

    """
    # Filter rows where 'first_name' or 'last_name' contain numbers
    rows_with_numbers = df[df['first_name'].str.contains(r'\d', regex=True) | df['last_name'].str.contains(r'\d', regex=True)]

    # Print the rows
    print("Rows where first_name or last_name contain numbers:")
    print(rows_with_numbers)

    # Take action based on the specified action parameter
    if action == 'remove':
        # Remove rows with numbers in 'first_name' or 'last_name' columns
        cleaned_df = df[~df.index.isin(rows_with_numbers.index)]
        print(f"Removed {len(rows_with_numbers)} rows with numbers in 'first_name' or 'last_name' columns.")
    elif action == 'replace':
        # Replace numeric values in 'first_name' and 'last_name' columns with an empty string
        df.loc[rows_with_numbers.index, ['first_name', 'last_name']] = ''
        cleaned_df = df
        print(f"Replaced numeric values in 'first_name' and 'last_name' columns.")
    elif action == 'remove_numbers':
        # Remove numeric values from 'first_name' and 'last_name' columns
        df['first_name'] = df['first_name'].str.replace(r'\d', '')
        df['last_name'] = df['last_name'].str.replace(r'\d', '')
        cleaned_df = df
        print(f"Removed numeric values from 'first_name' and 'last_name' columns.")

    return cleaned_df

In [9]:
def create_full_name_column(df):
    """
    Create a 'full_name' column by concatenating 'first_name' and 'last_name' columns,
    and remove the 'first_name' and 'last_name' columns.

    Parameters:
        df (pandas.DataFrame): The DataFrame to be modified.

    Returns:
        pandas.DataFrame: DataFrame with 'full_name' column created and 'first_name' and 'last_name' columns removed.

    """
    # Create 'full_name' column by concatenating 'first_name' and 'last_name'
    df['full_name'] = df['first_name'] + ' ' + df['last_name']

    # Remove 'first_name' and 'last_name' columns
    df.drop(columns=['first_name', 'last_name'], inplace=True)

    return df

In [10]:
def reorder_columns(df, desired_columns_order):
    """
    Reorder the columns of the DataFrame according to the desired order.

    Parameters:
        df (pandas.DataFrame): The DataFrame whose columns are to be reordered.
        desired_columns_order (list): The desired order of columns.

    Returns:
        pandas.DataFrame: DataFrame with columns reordered according to the desired order.

    """
    # Reorder the columns of the DataFrame
    df = df.reindex(columns=desired_columns_order)

    return df

In [11]:
def rename_columns(df, column_mapping):
    """
    Rename the columns of the DataFrame based on the provided mappings.

    Parameters:
    df (pd.DataFrame): The input DataFrame with original column names.

    Returns:
    pd.DataFrame: The DataFrame with renamed columns.

    """

    # Rename the columns
    df = df.rename(columns=column_mapping)

    return df

###Data Structure Configurations

In [12]:
# Define the desired order of columns
desired_columns_order = [
    'taxi_group_name', 'rank', 'full_name', 'vehicle_brand', 'driver_experience_group',
    'special_achievements_awarded', 'driver_endurance_score', 'driver_profitabilty_score',
    'driver_safety_adherence_score', 'driving_efficiency_score',
    'number_of_1_star_ratings', 'number_of_2_star_ratings', 'number_of_3_star_ratings',
    'number_of_4_star_ratings', 'number_of_5_star_ratings'
]

In [13]:
# Define the mapping from old column names to new column names
column_mapping = {
    'taxi_group_name': 'Taxi Group Name',
    'rank': 'Taxi Division Rank',
    'full_name': 'Driver Full Name',
    'vehicle_brand': 'Vehicle Brand',
    'driver_experience_group': 'Experience Group',
    'special_achievements_awarded': 'Special Awards',
    'driver_endurance_score': 'Endurance Score',
    'driver_profitabilty_score': 'Profitability Score',
    'driver_safety_adherence_score': 'Safety Score',
    'driving_efficiency_score': 'Efficiency Score',
    'number_of_1_star_ratings': '#of1Star Ratings',
    'number_of_2_star_ratings': '#of2Star Ratings',
    'number_of_3_star_ratings': '#of3Star Ratings',
    'number_of_4_star_ratings': '#of4Star Ratings',
    'number_of_5_star_ratings': '#of5Star Ratings'
}

In [14]:
numeric_columns = ['driver_endurance_score', 'driver_profitabilty_score', 'driver_safety_adherence_score',
                   'driving_efficiency_score']

In [15]:
# Define column data types
data_types = {
    'taxi_group_name': 'object',
    'rank': 'int64',
    'first_name': 'object',
    'last_name': 'object',
    'vehicle_brand': 'int64',
    'driver_experience_group': 'int64',
    'special_achievements_awarded': 'bool',
    'driver_endurance_score': 'float64',
    'driver_profitabilty_score': 'float64',
    'driver_safety_adherence_score': 'float64',
    'driving_efficiency_score': 'float64',
    'number_of_1_star_ratings': 'int64',
    'number_of_2_star_ratings': 'int64',
    'number_of_3_star_ratings': 'int64',
    'number_of_4_star_ratings': 'int64',
    'number_of_5_star_ratings': 'int64',
    'driver_id': 'object',
    'disabled': 'bool',
    'deleted': 'bool'
}

In [16]:
star_columns = ['number_of_1_star_ratings', 'number_of_2_star_ratings', 'number_of_3_star_ratings',
          'number_of_4_star_ratings', 'number_of_5_star_ratings']

###Execute

In [17]:
# Load the DataFrame from the pickle file
df = pd.read_pickle('data.pkl')

# Handle empty fields in the DataFrame
df = handle_empty_fields(df)

# Remove duplicate rows from the DataFrame
df = remove_duplicates(df)

# Convert columns to correct data types
df = convert_data_types(df, data_types)

# Replace negative values in star rating columns
df = replace_negative_values(df, star_columns)

# Flag outliers in numeric columns
df = flag_outliers(df, numeric_columns)

# Handle rows where first_name or last_name contain numbers
df = handle_rows_with_numbers(df)

# Create a full_name column by concatenating first_name and last_name, and remove original columns
df = create_full_name_column(df)

# Save the DataFrame to a pickle file before the columns are dropped and renamed (for further analysis)
df.to_pickle('data2.pkl')

# Reorder columns to the desired order
# Note: some fields are dropped
df = reorder_columns(df, desired_columns_order)

# Rename columns to the desired names
df = rename_columns(df, column_mapping)

# Save the final DataFrame to a pickle file
df.to_pickle('data3.pkl')

# Save the final DataFrame to a CSV file
df.to_csv('output.csv', index=False)

print("Data cleaning and transformation completed.")
print("Files saved as 'data2.pkl' for further analysis.")
print("The final DataFrame of this step saved as 'data3.pkl'.")
print("The final DataFrame saved as 'output.csv' as required by the project specifications.")

Duplicate rows: 794
Duplicate rows removed.
Negative values found in number_of_1_star_ratings
Negative values found in number_of_1_star_ratings:
    number_of_1_star_ratings
22                        -3
39                        -1
76                        -1
90                        -1
91                        -1
Negative values found in number_of_2_star_ratings
Negative values found in number_of_2_star_ratings:
     number_of_2_star_ratings
43                         -1
108                        -1
188                        -2
240                        -2
355                        -2
Negative values found in number_of_3_star_ratings
Negative values found in number_of_3_star_ratings:
      number_of_3_star_ratings
962                         -2
1025                        -1
1331                        -3
1407                        -2
1714                        -2
Negative values found in number_of_4_star_ratings
Negative values found in number_of_4_star_ratings:
      number

In [18]:
df.head()

,Taxi Group Name,Taxi Division Rank,Driver Full Name,Vehicle Brand,Experience Group,Special Awards,Endurance Score,Profitability Score,Safety Score,Efficiency Score,#of1Star Ratings,#of2Star Ratings,#of3Star Ratings,#of4Star Ratings,#of5Star Ratings
0,"Walsh, Hammond and Craig",0,Bruce Marshall,7,6,False,0.666398,-0.252875,0.279009,1.892210,9,38,40,20,22
1,"Walsh, Hammond and Craig",1,Carol Holmes,9,4,False,-1.177850,-1.560460,-1.211520,-0.510155,8,16,28,43,39
2,"Walsh, Hammond and Craig",1,Jay Bailey,16,6,False,-0.557925,0.083300,0.596134,0.257793,5,12,25,39,26
3,"Walsh, Hammond and Craig",1,Aimee Smith,3,4,False,-0.787186,-0.484124,0.966867,0.548890,8,8,16,52,34
4,"Walsh, Hammond and Craig",1,Hayley Hall,14,1,False,0.518161,-0.108167,-0.025077,-0.133834,22,32,53,16,13
